# 동국대 길찾기 & 학식 메뉴 알리미 챗봇

In [5]:
import gradio as gr
import pandas as pd
import networkx as nx
import difflib
import re
from PIL import Image
from datetime import datetime

# 데이터 불러오기
nodes = pd.read_csv("../data/DGU_place_final.csv")
edges = pd.read_csv('../data/DGU_route_final.csv')

# 그래프 구성
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(
        row["from"],
        row["to"],
        weight=row["time_min"] + row.get("extra_min", 0),
        time=row["time_min"],
        extra=row.get("extra_min", 0),
        has_steep_hill=row.get("has_steep_hill", 0),
        has_stairs=row.get("has_stairs", 0),
        has_signal=row.get("has_signal", 0),
        has_elevator=row.get("has_elevator", 0),
        description=row["description"]
    )

# ID ↔ 이름 매핑
id_to_name = nodes.set_index('id')['name'].to_dict()
name_to_id = {v: k for k, v in id_to_name.items()}
valid_names = list(name_to_id.keys())
nodes_dict = nodes.set_index("name")["Information"].to_dict()

# 이동 강도 분류 함수
def classify_intensity(hill_time, stair_time, total_time):
    ratio = (hill_time + stair_time) / total_time
    if ratio >= 0.6:
        return "🔴 상 (언덕/계단 포함 시간 비율이 60% 이상)"
    elif ratio >= 0.3:
        return "🟠 중 (30% 이상)"
    else:
        return "🟢 하 (30% 미만)"

def get_closest_name(name_input, candidates):
    match = difflib.get_close_matches(name_input, candidates, n=1, cutoff=0.6)
    return match[0] if match else None

def generate_path_response(start_name, end_name):
    corrected_start = get_closest_name(start_name, valid_names)
    corrected_end = get_closest_name(end_name, valid_names)

    if not corrected_start or not corrected_end:
        return f"❗️ '{start_name}' 또는 '{end_name}'은 인식할 수 없어요."

    start_id = name_to_id[corrected_start]
    end_id = name_to_id[corrected_end]

    if start_id not in G.nodes or end_id not in G.nodes:
        return f"❗️ '{corrected_start}' 또는 '{corrected_end}'은 그래프에 없어요."

    try:
        path = nx.shortest_path(G, source=start_id, target=end_id, weight='weight')
    except nx.NetworkXNoPath:
        return f"❗️ '{corrected_start}'에서 '{corrected_end}'까지 가는 길이 없어요."

    total_time = 0
    hill_time = 0
    stair_time = 0
    step_lines = []
    node_names = [id_to_name[n] for n in path]
    has_traffic_light = False
    has_elevator = False
    
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        edge = G[u][v]

        t = edge['time']
        extra = edge.get('extra', 0)
        segment_total = t + extra
        total_time += segment_total

        is_hill = edge.get('has_steep_hill', 0)
        is_stair = edge.get('has_stairs', 0)

        if is_hill and is_stair:
            hill_time += segment_total / 2
            stair_time += segment_total / 2
        elif is_hill:
            hill_time += segment_total
        elif is_stair:
            stair_time += segment_total

        # ✅ 신호등/엘리베이터 여부 확인
        if edge.get('has_signal', 0):
            has_traffic_light = True
        if edge.get('has_elevator', 0):
            has_elevator = True
        
        from_name = id_to_name[u]
        to_name = id_to_name[v]
        description = edge['description']
        step_lines.append(f"{i+1}. {from_name} → {to_name}:\n{description}")
        
        # 신호등/엘리베이터 여부 확인
        if "신호등" in description:
            has_traffic_light = True
        if "엘리베이터" in description:
            has_elevator = True
        

    intensity = classify_intensity(hill_time, stair_time, total_time)
    path_str = " → ".join(node_names)

    # 도착 건물 정보 조회
    info = nodes_dict.get(corrected_end)
    info_block = ""
    if info and isinstance(info, str) and info.strip():
        info_block = f"\n\n🏢 도착한 '{corrected_end}' 건물 정보:\n{info.strip()}"

    # ✅ 신호등 또는 엘리베이터가 있으면 주의 문구 추가
    caution = ""
    if has_traffic_light or has_elevator:
        caution = "\n\n⚠️ 경로 중 신호등 또는 엘리베이터 이용 시 대기 시간이 발생할 수 있습니다."
    
    return (
        f"📍 지금부터 '{corrected_start}'에서 '{corrected_end}'로 가는 길을 안내해줄게!\n\n"
        f"🚶 경로 설명:\n" + "\n".join(step_lines) + "\n\n"
        f"🧭 경로 요약:\n"
        f" - 총 경로: {path_str}\n"
        f" - 이동 강도: {intensity} "
        f"(언덕 포함 시간: {hill_time:.1f}분, 계단 포함 시간: {stair_time:.1f}분)\n"
        f" - 최소 예상 시간: {total_time:.1f}분"
        + info_block
        + caution
    )



# —————————— 학식 메뉴 로직 ——————————
menus = pd.read_csv("../data/DGU_menu_final.csv")
today = datetime.now()
default_date_str = f"{today.month:02d}월 {today.day:02d}일"

def query_menu(date_str, floor_query):
    # 상록원 해당 날짜 메뉴 필터
    df = menus[
        menus['restaurant'].str.contains("상록원") &
        (menus['date'] == date_str)
    ].copy()
    if df.empty:
        return f"❗️ {date_str} 상록원 학식 메뉴 정보가 없습니다."

    # floor, items 컬럼 생성
    df['floor'] = df['restaurant'].str.extract(r"상록원(\d+)층").astype(int)
    df['items'] = df['menu_clean'].str.split()

    # 1층 및 '일품코너'에서 제외할 단어 목록
    UNWANTED = {
        '삼겹', '돼지', '오스트리아산', '닭', '브라질산',
        '소', '미국산', '스팸', '외국산', '돈가스',
        '낙지', '베트남산', '쌀', '배추김치', '배추', '고춧가루'
    }

    # 특정 층만 조회
    if floor_query:
        fl = int(floor_query)

        # 1층: 메뉴만 나열하면서 unwanted 필터
        if fl == 1:
            df1 = df[df['floor'] == 1]
            if df1.empty:
                return f"❗️ {date_str} 1층 메뉴 정보가 없습니다."
            tokens = []
            for menu_str in df1['menu_clean']:
                tokens += [tok for tok in menu_str.split() if tok not in UNWANTED]
            return f"🍽️ {date_str} 상록원 1층 메뉴: {', '.join(tokens)}"

        # 2층 이상: 중식·석식별로, 같은 카테고리 한 줄 출력
        output = []
        for meal in ["중식", "석식"]:
            sub = df[(df['meal'] == meal) & (df['floor'] == fl)]
            if sub.empty:
                continue
            output.append(f"🍽️ {date_str} 상록원 {fl}층 {meal}")
            for cat, grp in sub.groupby('category'):
                items = sum(grp['items'].tolist(), [])
                # '일품코너' 항목에도 필터 적용
                if cat == '일품코너':
                    items = [tok for tok in items if tok not in UNWANTED]
                output.append(f"  • {cat}: {', '.join(items)}")
            output.append("")
        if not output:
            return f"❗️ {date_str} {fl}층 메뉴 정보가 없습니다."
        return "\n".join(output).rstrip()

    # 전체 층 조회
    output = []
    for meal in ["중식", "석식"]:
        sub_meal = df[df['meal'] == meal]
        if sub_meal.empty:
            continue
        output.append(f"🍽️ {date_str} 상록원 {meal}")
        for fl, grp_floor in sub_meal.groupby('floor'):
            output.append(f"  📍 {fl}층")
            for cat, grp_cat in grp_floor.groupby('category'):
                items = sum(grp_cat['items'].tolist(), [])
                # 1층이거나 '일품코너' 카테고리인 경우 필터 적용
                if fl == 1 or cat == '일품코너':
                    items = [tok for tok in items if tok not in UNWANTED]
                output.append(f"    • {cat}: {', '.join(items)}")
        output.append("")
    return "\n".join(output).rstrip()

# —————————— 통합 챗봇 함수 ——————————
MAP_PATH = "../images/dongguk_map.png"
def unified_chatbot(mode, query):
    if mode == "길찾기":
        m = re.search(r"(.*?)에서 (.*?)까지", query)
        if not m:
            return "예) '동대입구역에서 명진관까지' 형식으로 입력해주세요.", None
        result = generate_path_response(m.group(1).strip(), m.group(2).strip())
        return result, Image.open(MAP_PATH)

    # 학식 메뉴 모드
    # 1) 한글 “M월D일” 우선 매칭 (스페이스 0회 이상)
    kor = re.search(r"(\d{1,2})월\s*(\d{1,2})일", query)
    if kor:
        mm, dd = map(int, kor.groups())
        date_str = f"{mm:02d}월 {dd:02d}일"
    else:
        # 2) ISO “YYYY-MM-DD” 매칭
        iso = re.search(r"(\d{4})-(\d{1,2})-(\d{1,2})", query)
        if iso:
            mm, dd = int(iso.group(2)), int(iso.group(3))
            date_str = f"{mm:02d}월 {dd:02d}일"
        else:
            # 3) 둘 다 아니면 오늘
            today = datetime.now()
            date_str = f"{today.month:02d}월 {today.day:02d}일"

    # 4) 층수 파싱
    fm = re.search(r"(\d+)층", query)
    floor_q = fm.group(1) if fm else None

    return query_menu(date_str, floor_q),None

# —————————— Gradio Blocks 인터페이스 ——————————
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Row():
        with gr.Column(scale=3):  # 왼쪽 영역
            gr.Markdown("## 동국대학교 길찾기 & 학식 알리미 🤖")

        with gr.Column(scale=1):  # 오른쪽 영역
            gr.Image(value="../images/dongguk_logo.png",
                     width=100,
                     interactive=False,
                     show_label=False,
                     show_download_button=False,
                     show_fullscreen_button=False
                     )

    # ⬛ 초기 모드 선택 UI
    with gr.Column(visible=True) as mode_selector:
        gr.Markdown("### 사용할 기능을 선택해주세요 👇")
        chosen_mode = gr.Radio(
            choices=["길찾기", "학식 메뉴"],
            label="모드를 선택하세요",
        )
        start_btn = gr.Button("▶️ 시작")

    # 🟨 본 인터페이스 (초기에는 숨김)
    with gr.Column(visible=False) as main_interface:
        mode = gr.State()  # 선택된 모드를 저장하는 상태

        mode_display = gr.Markdown("")  # 선택된 모드 출력용
        with gr.Row():
            example1 = gr.Button("📍 동대입구역에서 법학관까지")
            example2 = gr.Button("🍱 3층 오늘의 학식 메뉴")
        query = gr.Textbox(
            lines=2,
            placeholder=(
                "길찾기     예) 동대입구역에서 명진관까지\n"
                "학식 메뉴  예) 3층 혹은 5월10일 2층"
            ),
            label="입력"
        )
        btn = gr.Button("전송")
        output_text = gr.Textbox(label="답변")
        output_img = gr.Image(label="캠퍼스 지도")
        reset_btn = gr.Button("🔁 모드 다시 선택하기")
        # 예시 버튼 동작
        example1.click(fn=lambda: "동대입구역에서 법학관까지", outputs=query)
        example2.click(fn=lambda: "3층", outputs=query)

        # 전송 버튼 동작
        btn.click(
            fn=unified_chatbot,
            inputs=[mode, query],
            outputs=[output_text, output_img]
        )

    # 🟩 시작 버튼 클릭 → UI 전환
    def init_mode_ui(selected_mode):
        if not selected_mode:
            return gr.update(visible=True), gr.update(visible=False), None, "⚠️ 모드를 선택해주세요."
        return (
            gr.update(visible=False),  # 모드 선택 화면 숨기기
            gr.update(visible=True),   # 본 UI 보여주기
            selected_mode,             # 상태로 모드 전달
            f"### ✅ 선택된 모드: **{selected_mode}**"
        )

    start_btn.click(
        fn=init_mode_ui,
        inputs=[chosen_mode],
        outputs=[mode_selector, main_interface, mode, mode_display]
    )
    def reset_mode_ui():
        return (
            gr.update(visible=True),   # 모드 선택 화면 보이기
            gr.update(visible=False),  # 메인 UI 숨기기
            None,                      # 상태 초기화
            ""                         # 모드 표시 초기화
        )

    reset_btn.click(
        fn=reset_mode_ui,
        inputs=[],
        outputs=[mode_selector, main_interface, mode, mode_display]
    )
    demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [3]:
pd.read_csv('../data/경로_주헌.csv').columns

Index(['from', 'to', 'distance_m', 'time_min', 'extra_min', 'has_crosswalk',
       'has_signal', 'has_stairs', 'has_steep_hill', 'has_elevator',
       'has_escalator', 'description'],
      dtype='object')

In [4]:
len(pd.read_csv('../data/경로_주헌.csv').columns)

12